# Aprendizaje Estadístico y Data Mining

## Práctica 1: Impacto de las restricciones de tráfico en la calidad del aire

### Objetivo
El 30 de diciembre de 2016 el Ayuntamiento de Madrid empieza a restringir el tráfico al centro de la ciudad
mediante la aplicación del escenario 3 del protocolo de contaminación. Esta fue la primera vez que se aplicó
un escenario tan restrictivo. Demuestra mediante visualizaciones de datos que ha ocurrido después de ello.
Para dicho problema, haremos uso de un dataset con datos sobre la calidad del aire en Madrid desde 2001. El
dataset contiene un archivo csv por año. Dentro de cada uno de ellos habrá medidas diarias de distintas
estaciones. Estas estaciones miden diferentes parámetros y no siempre todos se registran en cada una de las
estaciones.

* [Link al dataset.](https://www.kaggle.com/decide-soluciones/air-quality-madrid/downloads/csvs_per_year.zip/)

### Apartado 1

**Enunciado:** Cargar los archivos CSV en un único DataFrame, identificar y corregir posibles datos redundantes o anómalos, calcular la calidad del aire diario promediando los valores de las estaciones meteorológicas, estimar el primer valor según la tabla proporcionada, transformar los valores continuos a categóricos, almacenar la información en un DataFrame y guardarla en un archivo CSV, y finalmente determinar la calidad del aire general considerando el peor índice de las cinco partículas indicadas en la tabla.

<img src="./media/Enunciado%20Apartado%201.png" width="50%">

**Solución**

Cargamos los archivos CSV correspondientes a los datos de calidad del aire.

In [ ]:
%reset
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
import numpy as np
import seaborn as sns

ruta1_csvs = "data/csvs_per_year/csvs_per_year/"
ruta2_csvs = "data/stations.csv"

#Capturo todos los datos de Madrid desde 2001-2018 usando la librería glob para capturar los .csv
archivos_madrid_csv = glob.glob(os.path.join(ruta1_csvs, "*.csv"))
#Usando pandas, leemos y combinamos los archivos en un único Dataframe
df_madrid_sin_filtrar = pd.concat([pd.read_csv(archivo) for archivo in archivos_madrid_csv], ignore_index=True)


#Con el csv de stations, al ser uno solo, podemos leerlo y añadirlo directamente a otro Dataframe para diferenciar entre sets de datos.
df_stations = pd.read_csv(ruta2_csvs)

#Seleccionamos solo las columnas necesarias para medir la contaminación de cada día
columnas_necesarias = ['date', 'PM25', 'PM10', 'NO_2', 'O_3', 'SO_2', 'station']
#Declaramos también aquellas columnas que contienen únicamente las partículas necesarias para calcular la contaminación
columnas_calidad_aire = ['PM25', 'PM10', 'NO_2', 'O_3', 'SO_2']
#Filtramos el DataFrame para quedarnos solo con estas columnas
df_madrid = df_madrid_sin_filtrar[columnas_necesarias]

categoria_a_indice = {
    "Muy malo": 0,
    "Malo": 1,
    "Regular": 2,
    "Bueno": 3,
    "Muy bueno": 4
}

En primer lugar, verificaremos que porcentaje de datos nulos hay por columna, para decidir si borrarlas o mantenerlas.

In [ ]:
# Visualizamos el porcentaje de valores nulos por cada columna
print("PORCENTAJE DE VALORES NULOS POR COLUMNA DEL DF MADRID")
print((df_madrid.isnull().sum() / (df_madrid.count() + df_madrid.isnull().sum())) * 100)
df_madrid = df_madrid.dropna(subset=['PM25', 'PM10', 'NO_2', 'O_3', 'SO_2'], how='all')


A continuación, nos deshacemos de aquellas filas duplicadas que puedan aparecer:

In [ ]:
# Buscamos duplicados, y nos deshacemos de ellos para evitar redundancias
duplicados = df_madrid.duplicated().sum()
print("En este dataframe encontramos ", duplicados, " datos duplicados")
df_madrid = df_madrid.drop_duplicates()

Por último, habría que comprobar si existen datos recogidos de estaciones de las que no tengamos registro.

In [ ]:
print("Estaciones en df que no están en df_stations:")
estaciones_sin_registro = set(df_madrid["station"]) - set(df_stations["id"])

# Eliminamos aquellas filas cuya estación no aparezca en nuestro df de estaciones
df_madrid = df_madrid[~df_madrid["station"].isin(estaciones_sin_registro)]

# Por último, nos cercioraremos de que los datos de fechas estén en el formato deseado.
df_madrid['date'] = pd.to_datetime(df_madrid['date'])

Calcularemos la calidad del aire de cada día, agrupando los datos por fecha y estación, y obteniendo el peor dato de las 5 partículas para pasar el dato continuo a categórico.

In [ ]:
df_calidad_diario = df_madrid
# Encontramos el peor valor (o el maximo) de las 5 partículas contaminantes,
# y guardamos en otra columna el valor y de que partícula se trata
df_calidad_diario['calidad_cuantitativa'] = df_calidad_diario[columnas_calidad_aire].max(axis=1)
df_calidad_diario['particula_maxima'] = df_calidad_diario[columnas_calidad_aire].idxmax(axis=1)

Para categorizar, es decir, pasar de continuo a categórico, usaremos una función que podrá usar cualquier dataframe de aquí en adelante.

In [ ]:
# Esta función añadirá un campo extra de "calidad categórica" cualquier dataframe que contenga los campos
# "calidad_cuantitativa" y "partícula_maxima", siguiendo la tabla del enunciado.
def categorizar(df):
    condiciones = [
        (df['particula_maxima'] == 'PM25'),
        (df['particula_maxima'] == 'PM10'),
        (df['particula_maxima'] == 'NO_2'),
        (df['particula_maxima'] == 'O_3'),
        (df['particula_maxima'] == 'SO_2')
    ]

    # np select funciona mediante condiciones. Aplicando los rangos vistos en la imagen del enunciado que distamina las categorías, asignamos una calidad categórica a cada calidad_cuantitativa según la partícula máxima obtenida (de entre las 5).
    categorias = [
        np.select([df['calidad_cuantitativa'] <= 15, df['calidad_cuantitativa'] <= 30, df['calidad_cuantitativa'] <= 55, df['calidad_cuantitativa'] <= 110],
                  ['Muy bueno', 'Bueno', 'Regular', 'Malo'], default='Muy malo'),
        np.select([df['calidad_cuantitativa'] <= 25, df['calidad_cuantitativa'] <= 50, df['calidad_cuantitativa'] <= 90, df['calidad_cuantitativa'] <= 180],
                  ['Muy bueno', 'Bueno', 'Regular', 'Malo'], default='Muy malo'),
        np.select([df['calidad_cuantitativa'] <= 50, df['calidad_cuantitativa'] <= 100, df['calidad_cuantitativa'] <= 200, df['calidad_cuantitativa'] <= 400],
                  ['Muy bueno', 'Bueno', 'Regular', 'Malo'], default='Muy malo'),
        np.select([df['calidad_cuantitativa'] <= 60, df['calidad_cuantitativa'] <= 120, df['calidad_cuantitativa'] <= 180, df['calidad_cuantitativa'] <= 240],
                  ['Muy bueno', 'Bueno', 'Regular', 'Malo'], default='Muy malo'),
        np.select([df['calidad_cuantitativa'] <= 50, df['calidad_cuantitativa'] <= 100, df['calidad_cuantitativa'] <= 350, df['calidad_cuantitativa'] <= 500],
                  ['Muy bueno', 'Bueno', 'Regular', 'Malo'], default='Muy malo')
    ]

    # Asignamos el índice correspondiente a cada categoría, para normalizar los datos y podemos trabajar con valores numericos sencillos en los ejercicios posteriores.
    df['calidad_categorica'] = np.select(condiciones, categorias, default='Muy malo')
    df['indice'] = df['calidad_categorica'].map(categoria_a_indice)

    return df

Una vez definida la función que categorizará nuestros dataframes, la aplicamos al dataframe que contendrá la calidad del aire de todos los datos proporcionados por los csv.

In [ ]:
df_calidad_diario = categorizar(df_calidad_diario)

# Guardamos este dataframe en un csv, para analizarlo en caso de duda.
df_calidad_diario.to_csv('output/calidades_aire_diario.csv', index=False)

Una vez tenemos la partícula con mayor índice, comparamos entre estaciones que hayan recabado datos para cada día en concreto, para hallar la media de cada día.

In [ ]:
# Para ello, utilizaremos una función genérica, que será capaz de agrupar por x periodo (dias, meses, años, etc...)
#  y que encontrará la media para cada tipo de agrupación.
def calcular_medias_por_periodo(df, periodo):
    # Dependiendo del parámetro de periodo, agruparemos y haremos la media de los índices segun (dia, mes, año).
    df['date'] = df['date'].dt.to_period(periodo)
    df_resultado = df.groupby('date')['indice'].mean().reset_index()

    indice_a_categoria = {
        0: "Muy malo",
        1: "Malo",
        2: "Regular",
        3: "Bueno",
        4: "Muy bueno"
    }
    df_resultado['categoria_media'] = df_resultado['indice'].round().astype(int).map(indice_a_categoria)
    return df_resultado

In [ ]:
# Agrupando por días, creamos un dataframe que tiene en cuenta todos los datos que hay para cada día y obtiene la media general
df_calidad_diario['date'] = df_calidad_diario['date'].astype(str).pipe(pd.to_datetime)
df_media_max_por_dia = calcular_medias_por_periodo(df_calidad_diario, "D")

df_media_max_por_dia.to_csv('output/medias_por_dia.csv', index=False)

### Apartado 2

**Enunciado:** Visualizar la distribución de la calidad del aire en las categorías buena, admisible, mala y muy mala mediante un gráfico de tipo pie chart. Analizar la representación obtenida y extraer conclusiones sobre la proporción de cada categoría en los datos.

**Solución**

Usaremos una función genérica para graficar pie charts dado un dataframe cualquiera.

In [ ]:
def graficar_pie_chart(df):
    # Comenzamos obteniendo los datos que se representarán en el gráfico, que dependerá de las categorías de calidades obtenidas
    secciones = df['categoria_media'].value_counts()

    #Asignamos colores reconocibles a cada categoría, y sus respectivas etiquetas
    colores = ["#AAF683","#60D394","#FFD97D","darkred","#EE6055"]
    etiquetas = secciones.index
    if secciones.size == 4 :
        desfase = (0.05, 0.05, 0.07, 0.1)
    elif secciones.size == 3:
        desfase = (0.05, 0.05, 0.1)
    else:
        desfase = (0.05, 0.05, 0.07, 0.1, 0.1)

    # Con los datos del DataFrame de Calidades, graficamos el pie chart, asignándole un radio para obtener el pie chart deseado
    plt.figure(figsize=(8, 8))
    plt.pie(secciones, labels= etiquetas, autopct="%0.3f %%", colors=colores, explode=desfase)
    plt.axis('equal')
    plt.show()

Representaremos las distintas categorías de calidades de aire en un gráfico circular (pie chart). En este primer pie chart recogemos todos los datos disponibles desde 2001 hasta 2018, usando únicamente las medias categóricas de todas las calidades recogidas para cada día. En general, podemos observar como los datos son bastante positivos, resaltando que la calidad del aire "Muy mala" apenas llega al 0.01%.

In [ ]:
graficar_pie_chart(df_media_max_por_dia)

Este pie chart muestra las calidades de aire medias por cada día que se registraron desde inicios de 2001 hasta principios del 2018. Son datos en los que destaca la presencia mayoritaria de las calidades "Muy buena" y "Buena", ya que rondan el 80% de todos los datos recogidos. Sin embargo, que en el 20% de los días se hayan registrado calidades del aire contaminantes puede explicar el porqué se implantaron estas medidas tan restrictivas. Antes de concluir nada, habría que cerciorarnos de cómo eran los datos antes y después del día en que se implantaron dichas medidas.

### Apartado 3

**Enunciado:** Calcular la calidad media del aire para cada mes con el objetivo de evaluar la evolución de la contaminación tras la restricción de acceso al centro de Madrid. Representar visualmente los resultados mediante un diagrama de barras para analizar el impacto de las restricciones de tráfico. Comparar los valores antes y después de la implementación del protocolo y determinar si se observa una mejora en la calidad del aire.

**Solución**

In [ ]:
# Debido a que la función que agrupa por x tipo de periodo solo acepta datos no convertidos a Datetime, convertimos el campo date a str para evitar conflictos
df_media_max_por_dia['date'] = df_media_max_por_dia['date'].astype(str).pipe(pd.to_datetime)

# Agrupando por meses, y reutilizando estas dos funciones principales, conseguimos obtener la media que se produjo en cada mes desde 2001 hasta 2018
df_media_max_por_mes = calcular_medias_por_periodo(df_media_max_por_dia, "M")

# Por último, guardamos los resultados en un csv nuevo.
df_media_max_por_mes.to_csv('output/medias_por_mes.csv', index=False)

# Volvemos a convertir al tipo de date apropiado para evitar futuros conflictos con el tipo de dato
df_media_max_por_dia['date'] = df_media_max_por_dia['date'].astype(str).pipe(pd.to_datetime)


Filtramos ahora desde enero 2001 hasta diciembre 2016, y desde diciembre 2016 hasta abril 2018, para conocer con mas exactitud si las medidas tuvieron repercusiones positivas o insignificantes. Usaremos diagramas de barras, para observar la proporción de las categorías de calidad a lo largo de esos meses.

In [ ]:
def graficar_barras(df):
    colores = []
    for indice in df['indice']:
        if 0 <= indice < 0.5:
            colores.append('darkred')
        elif 0.5 <= indice < 2:
            colores.append('red')
        elif 2 <= indice < 3:
            colores.append('#FFB74D')
        elif 3 <= indice < 3.5:
            colores.append('#4CAF50')
        elif 3.5 <= indice < 4:
            colores.append('#64B5F6')

    df['date'] = df['date'].astype(str).pipe(pd.to_datetime)

    plt.figure(figsize=(16, 6))
    # Creamos el grafico de barras teniendo en cuenta los meses, y los índices medios que se obtuvieron en cada mes
    plt.bar(df['date'], df['indice'], color=colores, width=25)

    # En vez de poner en las etiquetas del eje x todos los meses, ponemos únicamente el año.
    plt.xticks(df['date'][::12], df['date'].dt.year[::12], rotation=45)

    plt.xlabel("Años")
    plt.ylabel("Índice Medio de Calidad del Aire")
    plt.title("Distribución de la Calidad del Aire por Mes")
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    plt.axvline(x=pd.Timestamp('2016-12-01'), color='black', linestyle='--', linewidth=2)

    plt.show()

graficar_barras(df_media_max_por_mes)


La mejoría es poco visible a primera vista. El primer grafo muestra valores de índices que oscilan entre 2.5(Regular-Bueno) y 3.5(Bueno-Muy bueno), por lo que no parecen datos preocupantes los obtenidos en el periodo en el que madrid estuvo sin restricciones de acceso. Debemos tener en cuenta que en Madrid existen actualmente 1,4 millones de coches activos, lo que son aproximadamente 1.07 coches por vivienda. Estos valores nos dicen que, para la cantidad de personas que pueden conducir que hay en Madrid (2,22 millones), no son tantos vehículos.
Además, para determinar si verdaderamente fueron necesarias dichas medidas, deberíamos tener en cuenta también la cantidad de coches que entran diariamente a Madrid Centro. 
Teniendo en cuenta únicamente los datos disponibles, podemos afirmar que si fueron beneficiosas para el medio ambiente, pero no es que hubiese un problema gravísimo de contaminación que pidiese la instauración de estas medidas tan extremas.

### Apartado 4

**Enunciado:** Generar un diagrama de cajas donde cada caja represente la calidad del aire en un año distinto para visualizar la evolución de la contaminación a lo largo del tiempo. Analizar la dispersión, los valores atípicos y la tendencia general de los datos, interpretando si ha habido mejoras o empeoramientos en la calidad del aire en los últimos años.

**Solución**

A partir del dataframe de medias de calidad por mes, contruimos un diagrama de cajas en el que cada cada agrupe la información de 12 meses, para poder ver los cambios de un año para otro.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def graficar_boxplot(df):
    plt.figure(figsize=(14, 7))

    sns.set_theme(style="whitegrid")

    # El boxplot nos mostrará toda la información relativa a las estadisticas de cada año, dependiendo de los indices d ecada mes
    sns.boxplot(x="Año", y="indice", data=df,
                          palette="coolwarm", width=0.6, linewidth=1)

    # Utilizamos un swarmplot para visualizar mejor los puntos, y entender el boxplot.
    sns.swarmplot(x="Año", y="indice", data=df, size=3, color="purple", alpha=0.8)

    plt.title("Distribución de la Calidad del Aire por Año", fontsize=16, weight='bold')
    plt.suptitle("", fontsize=12)
    plt.xlabel("Año", fontsize=14, weight='bold')
    plt.ylabel("Media de Calidad del Aire", fontsize=14, weight='bold')

    # Asignamos las etiquetas correspondientes a los ejes
    plt.xticks(rotation=45, fontsize=12)
    plt.yticks(ticks=[1, 2, 3, 4, 5], labels=["Muy malo", "Malo", "Regular", "Bueno", "Muy bueno"], fontsize=12)

    plt.grid(True, linestyle='--', alpha=0.6)

    plt.tight_layout()

    plt.show()

df_media_max_por_mes["Año"] = df_media_max_por_mes["date"].dt.year

graficar_boxplot(df_media_max_por_mes)

En el periodo anterior a las medidas de acceso, los datos oscilaban en un rango bastante grande, aunque cabe decir que seguían siendo valores bastante "poco contaminantes". En 2006 Madrid experienció la contaminación alta, ya que la mayoría de las categorías se encontraban entre Regular y Malo, pero aún así, se esperó hasta 2016 para implantar las medidas. Los valores obtenidos después del 30/12/2016 oscilan en un rango mucho menor, sobre todo si comparamos el año 2017 con los demás. Indica que las medidas surtieron efecto positivo, pero que nunca han sido del todo necesarias teniendo como referencia las buenas medias de calidad de aire entre los años 2001 y 2016. De hecho, antes del año de la instauración de las medidas (2008 - 2016), los datos eran bastante optimistas, lo que pone en duda si alguna vez fueron necesarias de verdad.
No hay más que echar un ojo a un artículo de Septiembre 2024, donde se publicó que la Justicia tumbó las medidas de bajas emisiones de Madrid Centro a petición de Vox. son datos que confirman la teoría que acabamos de elaborar.

### 5. Bibliografía

- Código para tratar valores nulos: 
  https://es.stackoverflow.com/questions/279443/c%C3%B3mo-calcular-el-recuento-de-datos-faltantes-para-columnas
- Código para generar un pie chart personalizado: 
  https://numython.github.io/posts/graficas-de-pastel-con-matplotlib/
- Código para generar un boxplot con el swarmplot combinado: 
  https://www.datacamp.com/es/tutorial/python-boxplots?dc_referrer=https%3A%2F%2Fwww.google.com%2F
- Artículo sobre la intensidad de tráfico en Madrid: 
  https://www.comunidad.madrid/servicios/transporte/estudio-intensidad-media-diaria-vehiculos-imd https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=4a2e4207bb864410VgnVCM1000000b205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD
- Artículo "La justicia tumba las zonas de bajas emisiones de Madrid a petición de VOX": 
  https://climatica.coop/justicia-anula-zonas-bajas-emisiones-madrid/